In [0]:
SELECT * FROM bronze_db.taxi_zone LIMIT 10;

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


In [0]:
CREATE DATABASE IF NOT EXISTS silver_db


In [0]:
USE silver_db;
SELECT current_database();

current_schema()
silver_db



- passenger_count > 0
- trip_distance > 0
- total_amount > 0
- Date et heure de demarrage doit être antérieure à date et heure de fin du trajet
- Considérer unique les payements en carte de crédit


## 1. Preparer la table silver si elle n'existe pas

In [0]:
CREATE TABLE IF NOT EXISTS processed_trips
USING DELTA
AS
SELECT *
FROM bronze_db.raw_trips
WHERE
  passenger_count > 0
  AND trip_distance > 0
  AND total_amount > 0
  AND tpep_pickup_datetime < tpep_dropoff_datetime
  AND tip_amount >= 0
  AND payment_type = 1;



num_affected_rows,num_inserted_rows


## 2. Charger seulement les nouvelles données de la table bronze

In [0]:
WITH new_data AS (
  SELECT *
  FROM bronze_db.raw_trips b
  WHERE b.ingestion_timestamp > 
  (SELECT max(ingestion_timestamp) FROM processed_trips)

AND b.passenger_count > 0
AND b.trip_distance > 0
AND b.total_amount > 0
AND b.tpep_pickup_datetime < b.tpep_dropoff_datetime
AND b.tip_amount >= 0
AND b.payment_type = 1
)

MERGE INTO processed_trips s
USING new_data n
ON s.file_name = n.file_name
AND s.tpep_pickup_datetime = n.tpep_pickup_datetime
AND s.tpep_dropoff_datetime = n.tpep_dropoff_datetime
AND s.ingestion_timestamp = n.ingestion_timestamp
WHEN MATCHED THEN UPDATE SET *
WHEN NOT MATCHED THEN INSERT *;


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
2674590,0,0,2674590


In [0]:
SELECT count(*) FROM processed_trips

count(1)
12292364


In [0]:
SELECT DISTINCT file_name FROM processed_trips ORDER BY file_name

file_name
yellow_tripdata_2024-01.parquet
yellow_tripdata_2024-02.parquet
yellow_tripdata_2024-03.parquet
yellow_tripdata_2024-04.parquet
yellow_tripdata_2024-05.parquet
